# AWS 트레이니움(Trainium)에서 Llama 모델 미세 조정하기

이번 실습에서는 AWS 트레이니움(Trainium)에서 [Llama 2](https://huggingface.co/meta-llama/Llama-2-70b-hf)와 같은 오픈 LLM을 미세 조정하는 방법을 배웁니다. 예제에서는 허깅페이스 최적의 뉴런(Optimum Neuron), [Transformers](https://huggingface.co/docs/transformers/index), 그리고 데이터 세트를 활용합니다.

이번 실습을 통해 다음을 배우게 됩니다:

1. [AWS 환경 설정](#1-aws-환경-설정)
2. [데이터 세트 불러오기 및 준비](#2-데이터-세트-불러오기-및-준비)
3. [AWS Trainium을 사용하여 `NeuronTrainer`로 Llama 모델 미세 조정하기](#3-aws-trainium을-사용하여-neurontrainer로-llama-모델-미세-조정하기)
4. [미세 조정된 Llama 모델 평가 및 테스트](#4-미세-조정된-llama-모델-평가-및-테스트)

---

## 간단한 소개: AWS Trainium

[AWS Trainium (Trn1)](https://aws.amazon.com/ko/ec2/instance-types/trn1/?nc1=h_ls)은 딥 러닝(DL) 훈련 작업을 위해 특별히 제작된 EC2 인스턴스입니다. Trainium은 고성능 훈련 작업에 중점을 둔 [AWS Inferentia](https://aws.amazon.com/ko/ec2/instance-types/inf1/?nc1=h_ls)의 후속 제품입니다. Trainium은 자연어 처리, 컴퓨터 비전, 추천 모델을 훈련하는데 최적화되어 있습니다. 이 가속기는 FP32, TF32, BF16, FP16, UINT8 및 구성 가능한 FP8을 포함한 다양한 데이터 유형을 지원합니다.

가장 큰 Trainium 인스턴스인 `trn1.32xlarge`는 500GB 이상의 메모리를 제공하여 약 100억 개의 파라미터 모델을 단일 인스턴스에서 쉽게 미세 조정할 수 있습니다. 아래에서 사용 가능한 인스턴스 유형에 대한 개요를 확인할 수 있습니다. 더 자세한 내용은 [여기](https://aws.amazon.com/ko/ec2/instance-types/trn1/?nc1=h_ls)에서 확인할 수 있습니다:

| 인스턴스 크기     | 엑셀러레이터 | 엑셀러레이터 메모리 (GB) | vCPU | 인스턴스 메모리 (GiB) | 시간당 요금 (미국 달러, USD) |
|-------------|--------|-----------------|------|----------------|---------------------|
| trn1.2xlarge | 1      | 32              | 8    | 32             | 1.34                |
| trn1.32xlarge| 16     | 512             | 128  | 512            | 21.50               |
| trn1n.32xlarge| 16     | 512             | 128  | 512            | 24.78               |


---
 
*참고: 이번 실습은 trn1.32xlarge AWS EC2 인스턴스에서 수행했습니다.*


## 1. AWS 환경 설정

이번 예제에서는 32개의 뉴런 코어와 16개의 가속기를 포함한 `trn1.32xlarge` 인스턴스와 [허깅페이스 뉴런 딥러닝(Hugging Face Neuron Deep Learning)](https://aws.amazon.com/marketplace/pp/prodview-gr3e6yiscria2) AMI를 사용합니다. 해당 허깅페이스 AMI에는 Transformers, Datasets, Optimum, Neuron 패키지 같이 중요한 라이브러리가 미리 설치되어 있어 환경 관리 할 필요 없이 매우 쉽게 시작할 수 있습니다.

이번 실습에서는 인스턴스를 생성하는 방법을 다루지 않습니다. 환경 설정에 대한 단계별 가이드가 포함된 [“허깅페이스 Transformers를 위한 AWS Trainium 설정하기”](https://www.philschmid.de/setup-aws-trainium)에 대한 블로그 포스트에서 해당 내용을 확인할 수 있습니다.

인스턴스를 설정하고 실행하면 SSH로 접속할 수 있습니다. 하지만 터미널 내에서 개발하는 대신 데이터 세트 준비와 훈련 시작을 위해 `쥬피터(Jupyter)` 환경을 사용하고자 합니다. 이를 위해 `ssh` 명령어에 포트를 추가하여 포워딩하고 로컬 호스트 트래픽을 Trainium 인스턴스로 터널링해야 합니다.

```bash
PUBLIC_DNS="" # IP 주소, 예를 들어. ec2-3-80-....
KEY_PATH="" # 키 보관 경로, 예를 들어. ssh/trn.pem

ssh -L 8080:localhost:8080 -i ${KEY_NAME}.pem ubuntu@$PUBLIC_DNS
```

이제 [예제 노트북과 스크립트](https://github.com/huggingface/optimum-neuron/tree/main/notebooks/text-generation)가 포함된 Optimum 저장소를 가져옵니다.

```bash
git clone https://github.com/huggingface/optimum-neuron.git
```

다음으로 디렉토리를 `notebooks/text-generation`으로 변경하고 `쥬피터` 환경을 실행합니다.


```bash
# 경로 변경
cd optimum-neuron/notebooks/text-generation
# jupyter 실행
python -m notebook --allow-root --port=8080
```

**`쥬피터`** 의 출력과 함께 노트북으로 연결되는 URL이 익숙하게 보일 것입니다.

**`http://localhost:8080/?token=8c1739aff1755bd7958c4cfccc8d08cb5da5234f61f129a9`**

이 링크를 클릭하면 **`쥬피터`** 환경이 로컬 브라우저에서 열립니다. `llama2-7b-fine-tuning.ipynb` 노트북을 열고 시작해 봅시다.

_참고: 쥬피터 환경을 데이터 세트 준비에만 사용할 것이며 그 후 `torchrun`을 사용하여 분산 훈련을 위한 훈련 스크립트를 실행합니다._

In [ ]:
%pip install optimum-neuron torch-neuronx

공식 Llama 2 체크포인트를 사용하려면 모델에 접근할 수 있는 허깅페이스 계정에 로그인하고 게이트된 저장소에 접근할 수 있는 토큰을 사용해야 합니다. 다음 명령어를 실행하여 이 작업을 수행할 수 있습니다:

_참고: 게이트 없는 체크포인트도 제공합니다._

In [ ]:
# !huggingface-cli login --token 토큰_입력

## 2. 데이터 세트 불러오기 및 준비

[InstructGPT 논문](https://arxiv.org/abs/2203.02155)에 설명된 여러 행동 카테고리(브레인스토밍, 분류, 폐쇄형 QA, 생성, 정보 추출, 개방형 QA, 요약) 분야에 수천 명의 Databricks 직원들이 생성한 명령어-응답 항목들을 포함한 오픈 소스 데이터 세트 [dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k)를 사용합니다.

```python
{
  "instruction": "World of Warcraft란 무엇인가?",
  "context": "",
  "response": "World of Warcraft는 대규모 온라인 멀티 플레이어 롤플레잉 게임입니다. 2004년에 Blizzard Entertainment에서 출시했습니다."
}
```

`samsum` 데이터 세트를 불러오기 위해 허깅페이스 Datasets 라이브러리의 `load_dataset()` 메소드를 사용합니다.

In [ ]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# dataset size: 15011


모델을 명령어로 조정하려면 구조화된 예제들을 명령어로 표현된 작업들의 모음으로 변환해야 합니다. 샘플을 입력 받아서 지정된 형식의 명령어 문자열을 반환하는 메소드 `formatting_function`을 정의합니다.

In [ ]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt


포맷팅 함수를 임의의 예제로 테스트 합니다.

In [ ]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

추가로, 샘플을 포맷하는 것 외에 더 효율적인 훈련을 위해 여러 샘플을 하나의 시퀀스로 묶는 작업을 수행합니다. 이는 여러 샘플을 하나의 시퀀스로 스택하고 EOS 토큰으로 구분하는 것을 의미합니다. 이렇게 하면 훈련이 더 효율적입니다. 샘플을 패킹/스태킹하는 작업은 훈련 전 또는 훈련 중에 수행할 수 있습니다. 이번 실습에서는 시간 절약을 위해 훈련 전에 이 작업을 수행합니다. 데이터 세트와 패킹 함수를 받아 패킹된 데이터 세트를 반환하는 유틸리티 메서드 [pack_dataset](../scripts/utils/pack_dataset.py)을 만들었습니다.

In [ ]:
from transformers import AutoTokenizer

# Hugging Face model id
#model_id = "NousResearch/Llama-2-7b-hf"
model_id = "philschmid/Llama-2-7b-hf" # ungated
# model_id = "meta-llama/Llama-2-7b-hf" # gated

tokenizer = AutoTokenizer.from_pretrained(model_id)

데이터 세트를 패킹/스태킹하려면 먼저 토큰화한 다음 `pack_dataset` 메서드를 사용하여 패킹할 수 있습니다. 데이터를 준비하기 위해 다음을 수행합니다:

1. 템플릿 메서드를 사용하여 샘플을 포맷하고 각 샘플의 끝에 EOS 토큰을 추가합니다.
2. 데이터 세트를 텍스트에서 토큰으로 변환하기 위해 토큰화합니다.
3. 데이터 세트를 2048개의 토큰으로 패킹합니다.

In [ ]:
from random import randint
# add utils method to path for loading dataset
import sys
sys.path.append("./scripts/utils") # make sure you change this to the correct path 
from pack_dataset import pack_dataset


# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# print random sample
print(dataset[randint(0, len(dataset))]["text"])

# tokenize dataset
dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
)

# chunk dataset
lm_dataset = pack_dataset(dataset, chunk_length=2048) # We use 2048 as the maximum length for packing

데이터 세트를 처리한 후 디스크에 저장합니다. 나중에 사용하기 위해 S3 또는 허깅페이스 허브에 저장할 수도 있습니다.

_참고: 데이터세트를 패킹하고 전처리하는 작업은 Trainium 인스턴스 외부에서 실행할 수 있습니다._

In [ ]:
# 훈련 데이터 세트 디스크에 저장
dataset_path = "tokenized_dolly"
lm_dataset.save_to_disk(dataset_path)

## 3. AWS Trainium을 사용하여 `NeuronTrainer`로 Llama 모델 미세 조정하기

보통은 **[Trainer](https://huggingface.co/docs/transformers/v4.19.4/en/main_classes/trainer#transformers.Trainer)**와 **[TrainingArguments](https://huggingface.co/docs/transformers/v4.19.4/en/main_classes/trainer#transformers.TrainingArguments)**를 사용하여 PyTorch 기반의 트랜스포머 모델을 미세 조정합니다.

하지만 AWS Trainium 인스턴스에서 훈련할 때는 성능, 견고성 및 안전성을 향상 시킬 수 있는 `NeuronTrainer`를 사용합니다. `NeuronTrainer`는 `optimum-neuron` 라이브러리의 일부로 `Trainer`를 1:1로 대체할 수 있습니다.

AWS Trainium에서 분산 훈련 할 때는 몇 가지 주의사항이 있습니다. Llama는 큰 모델이기 때문에 단일 가속기에 맞지 않을 수 있습니다. 그래서 `NeuronTrainer`에 다양한 분산 훈련 전략을 지원하고 있습니다. 다음 훈련 전략이 포함됩니다:

* [ZeRO-1](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/frameworks/torch/torch-neuronx/tutorials/training/zero1_gpt2.html): 옵티마이저 상태를 여러 장치에 분할합니다.
* [Tensor Parallelism](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/neuronx-distributed/tensor_parallelism_overview.html):모델 파라미터를 여러 장치의 지정된 차원에 따라 분할하며 `tensor_parallel_size`로 정의합니다. 
* [Sequence parallelism](https://arxiv.org/pdf/2205.05198.pdf): 텐서 병렬 영역 외부의 시퀀스 축에서 활성화를 분할합니다. 메모리를 절약하면서 활성화를 분할하는데 유용합니다.
* [Pipeline Parallelism](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/neuronx-distributed/pipeline_parallelism_overview.html): _지원 예정_

`optimum-neuron` 라이브러리는 이미 이러한 분산 훈련 전략을 구현한 `run_clm.py`를 준비해놨습니다. 라이브러리에서 지원하는 분산 훈련 전략은 [공식 문서](https://huggingface.co/docs/optimum-neuron/guides/distributed_training)에서 확인할 수 있습니다. AWS 가속기에서는 모델을 훈련할 때 먼저 훈련 인수와 함께 모델을 컴파일해야 합니다.

이를 극복하기 위해 해당 라이브러리에 [모델 캐시](https://huggingface.co/docs/optimum-neuron/guides/cache_system)를 포함되어 있습니다. 이를 통해 허깅페이스 허브에서 미리 컴파일된 모델과 구성을 사용하여 컴파일 단계를 건너뛸 수 있습니다. 하지만 구성에서 변경이 있을 경우 새로운 컴파일이 발생할 수 있으며, 이는 일부 캐시 누락으로 이어질 수 있습니다.

_참고: 구성 파일이 캐시에 없으면 라이브러리 [Github](https://github.com/huggingface/optimum-neuron/issues)에 이슈를 열어 주세요. 구성 파일 추가에 도움 줄 수 있습니다._

이미 훈련을 위한 구성을 미리 컴파일했습니다. 따라서 아래 셀을 건너뛰거나 다시 실행하더라도 캐시된 구성을 재사용하기 때문에 몇 분 정도만 소요될 것입니다.

In [ ]:
%%bash
apt-get install apt-utils gnupg  -y

# Configure Linux for Neuron repository updates
. /etc/os-release
tee /etc/apt/sources.list.d/neuron.list > /dev/null <<EOF
deb https://apt.repos.neuron.amazonaws.com ${VERSION_CODENAME} main
EOF
wget -qO - https://apt.repos.neuron.amazonaws.com/GPG-PUB-KEY-AMAZON-AWS-NEURON.PUB | apt-key add -

### 설정

---

필요한 패키지를 설치하고 업그레이드합니다. 아래 셀을 처음 실행한 후 쥬피터 노트북 커널을 재시작하세요.

---

In [ ]:
%pip config set global.extra-index-url https://pip.repos.neuron.amazonaws.com

In [ ]:
!apt-get update -y

In [ ]:
!apt install -y aws-neuronx-dkms=2.*

In [ ]:
%%bash

# Update OS packages 
#apt-get update -y

# Install OS headers 
#apt-get install linux-headers-$(uname -r) -y

# Install git 
#apt-get install git -y

# install Neuron Driver
apt-get install aws-neuronx-dkms=2.* -y

# Install Neuron Runtime 
apt-get install aws-neuronx-collectives=2.* -y
apt-get install aws-neuronx-runtime-lib=2.* -y

# Install Neuron Tools 
apt-get install aws-neuronx-tools=2.* -y

# Add PATH
export PATH=/opt/aws/neuron/bin:$PATH

In [ ]:
%pip install -U neuronx-cc==2.* torch-neuronx torch

In [ ]:
# precompilation command 
!MALLOC_ARENA_MAX=64 neuron_parallel_compile torchrun --nproc_per_node=32 scripts/run_clm.py \
 --model_id {model_id} \
 --dataset_path {dataset_path} \
 --bf16 True \
 --learning_rate 5e-5 \
 --output_dir dolly_llama \
 --overwrite_output_dir True \
 --skip_cache_push True \
 --per_device_train_batch_size 1 \
 --gradient_checkpointing True \
 --tensor_parallel_size 8 \
 --max_steps 10 \
 --logging_steps 10 \
 --gradient_accumulation_steps 16

_참고: 캐시 없이 컴파일하는 데 약 40분이 소요될 수 있습니다. 컴파일 중에 `dolly_llama_sharded`에 임시 파일이 생성되므로 나중에 제거해야 합니다. 또한 잠재적인 충돌을 피하기 위해 CPU 할당을 제한하는 `MALLOC_ARENA_MAX=64`를 추가해야 하므로 이 값을 지금은 제거하지 마세요._

In [ ]:
# remove dummy artifacts which are created by the precompilation command
!rm -rf dolly_llama

컴파일이 완료되면 비슷한 명령어로 훈련을 시작할 수 있습니다. 다만 `neuron_parallel_compile`을 제거해야 합니다. `torchrun`을 사용하여 훈련 스크립트를 실행합니다. `torchrun`은 PyTorch 모델을 여러 가속기에서 자동으로 분산시키는 도구입니다. `nproc_per_node` 인수로 가속기의 수를 하이퍼파라미터와 함께 전달할 수 있습니다. 컴파일 명령어와의 차이점은 `max_steps=10`에서 `num_train_epochs=3`으로 변경한 것입니다.

다음 명령어를 사용하여 훈련을 시작합니다.

In [ ]:
!MALLOC_ARENA_MAX=64 torchrun --nproc_per_node=32 scripts/run_clm.py \
 --model_id {model_id} \
 --dataset_path {dataset_path} \
 --bf16 True \
 --learning_rate 5e-5 \
 --output_dir dolly_llama \
 --overwrite_output_dir True \
 --per_device_train_batch_size 1 \
 --gradient_checkpointing True \
 --tensor_parallel_size 8 \
 --num_train_epochs 3 \
 --logging_steps 10 \
 --gradient_accumulation_steps 16

이렇게 해서 AWS Trainium에서 Llama 7B 모델을 성공적으로 훈련했습니다. dolly 데이터 세트(15,000개의 샘플)로 3 에포크 동안 훈련하는데 총 43분 24초가 소요되었으며 실제 훈련 시간은 31분 46초에 불과했습니다. 이는 trn1.32xlarge 인스턴스에서 전체 훈련을 수행하는 데 약 $15.5의 비용이 든다는 것을 의미합니다. 나쁘지 않네요!

하지만 모델을 공유하고 테스트하기 전에 모델을 통합해야 합니다. 훈련 중에 텐서 병렬 처리를 사용했기 때문에 모델을 사용하기 전에 모델 가중치를 통합해야 합니다. 텐서 병렬 처리는 모델 가중치를 여러 작업자에 걸쳐 분할하며 훈련 중에는 분할된 체크포인트만 저장됩니다.

Optimum CLI는 `optimum neuron consolidate` 명령어를 통해 이를 매우 쉽게 수행할 수 있는 방법을 제공합니다:

In [ ]:
!optimum-cli neuron consolidate dolly_llama/tensor_parallel_shards dolly_llama

이미 safetensors로 통합했으므로 "분할된" 체크포인트를 제거합시다.

In [ ]:
!rm -rf dolly_llama/tensor_parallel_shards

## 4. 미세 조정된 Llama 모델 평가 및 테스트

훈련과 마찬가지로 AWS Trainium 또는 AWS Inferentia2에서 추론을 실행하려면 올바르게 사용할 수 있도록 모델을 컴파일해야 합니다. 이번 실습에서는 추론 테스트를 위해 Trainium 인스턴스를 사용하지만 보통 추론에는 Inferentia2를 사용하는게 좋습니다.

Optimum Neuron은 간단한 추론을 위해 Transformers의 AutoModel 클래스와 유사하게 구현되었습니다. `NeuronModelForCausalLM` 클래스를 사용하여 기본 트랜스포머 체크포인트를 로드하고 이를 뉴런 모델로 변환합니다. 

In [ ]:
from optimum.neuron import NeuronModelForCausalLM
from transformers import AutoTokenizer

compiler_args = {"num_cores": 2, "auto_cast_type": 'fp16'}
input_shapes = {"batch_size": 1, "sequence_length": 2048}

tokenizer = AutoTokenizer.from_pretrained("dolly_llama")
model = NeuronModelForCausalLM.from_pretrained(
        "dolly_llama",
        export=True,
        **compiler_args,
        **input_shapes)


_참고: 추론 컴파일에는 약 25분이 소요될 수 있습니다. 다행히도 이 작업은 한 번만 수행하면 됩니다. 나중에 모델을 저장할 수 있기 때문입니다. Inferentia2에서 실행하려면 다시 컴파일해야 합니다. 컴파일은 매개변수와 하드웨어에 따라 달라집니다._

In [ ]:
# COMMENT IN if you want to save the compiled model
# model.save_pretrained("compiled_dolly_llama")

이제 추론을 테스트할 수 있지만 미세 조정을 위해 사용한 프롬프트 형식에 맞게 입력의 형식을 변경해야 합니다. 이를 위해 `instruction`과 선택적으로 `context`가 포함된 `dict`를 인자로 받는 메소드를 만듭니다.

In [ ]:
def format_dolly_infernece(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if "context" in sample else None
    response = f"### Answer\n"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt


def generate(sample): 
    prompt = format_dolly_infernece(sample)
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs,
                         max_new_tokens=512,
                         do_sample=True,
                         temperature=0.9,
                         top_k=50,
                         top_p=0.9)
    return tokenizer.decode(outputs[0], skip_special_tokens=False)[len(prompt):]

이제 추론을 테스트해 보겠습니다. 먼저 컨텍스트 없이 테스트해 보겠습니다.

_참고: AWS Trainium에서 2개의 코어를 사용하여 추론할 때 매우 빠르게 수행될 것을 기대하지 마세요. 추론에는 Inferentia2를 사용하는 것이 좋습니다._

In [ ]:
prompt = {
  "instruction": "Can you tell me something about AWS?"
}
res = generate(prompt)

print(res)

> AWS stands for Amazon Web Services. AWS is a suite of remote computing services offered by Amazon. The most widely used of these include Amazon Elastic Compute Cloud (Amazon EC2), which provides resizable compute capacity in the cloud; Amazon Simple Storage Service (Amazon S3), which is an object storage service; and Amazon Elastic Block Store (Amazon EBS), which is designed to provide high performance, durable block storage volumes for use with AWS instances. AWS also provides other services, such as AWS Identity and Access Management (IAM), a service that enables organizations to control access to their AWS resources, and AWS Key Management Service (AWS KMS), which helps customers create and control the use of encryption keys.</s>

올바르게 작동하는 것 같습니다. 이제 RAG 애플리케이션처럼 일부 컨텍스트를 추가해 보겠습니다.

In [ ]:
prompt = {
  "instruction": "How can train models on AWS Trainium?",
  "context": "🤗 Optimum Neuron is the interface between the 🤗 Transformers library and AWS Accelerators including [AWS Trainium](https://aws.amazon.com/machine-learning/trainium/?nc1=h_ls) and [AWS Inferentia](https://aws.amazon.com/machine-learning/inferentia/?nc1=h_ls). It provides a set of tools enabling easy model loading, training and inference on single- and multi-Accelerator settings for different downstream tasks."
}
res = generate(prompt)

print(res)

> You can use the Optimum Neuron interface to train models on AWS Trainium.</s> 

멋지네요. 모델이 제공된 컨텍스트를 정확히 사용하고 있습니다. 여기까지입니다. AWS Trainium에서 Llama를 미세 조정한 것을 축하드립니다.